In [106]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# for display progress
from ipywidgets import IntProgress
from IPython.display import display

In [69]:
# to store hrefs of hotels
hrefs = []

In [70]:
# initialize web driver
driver = webdriver.Chrome()

In [71]:
wait = WebDriverWait(driver, 10)

In [72]:
# fill the url of the first page of hotel list
driver.get("https://www.tripadvisor.co.uk/Hotels-g297418-Zhuhai_Guangdong-Hotels.html")

In [81]:
while True:
    # roll down to the bottom
    # here 5300 is a special value for this page, need change for other pages
    js= "var q=document.body.scrollTop=5300"
    driver.execute_script(js)
    
    # extract html and get hrefs
    soup = bs4.BeautifulSoup(driver.page_source, "lxml")
    hotel_table = soup.find("div", {"id": "taplc_hotels_list_short_cells2_0"})
    hotel_list = hotel_table.findAll("div", {"class": "listing easyClear  p13n_imperfect "})
    
    for hotel in hotel_list:
        hrefs.append(hotel.find("a", {"class": "property_title "})['href'])
        
    try:
        element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".nav.next.ui_button.primary.taLnk")))
        time.sleep(3)
        element.click()
    except:
        break

In [82]:
driver.close()

In [83]:
hrefs = list(set(hrefs))

In [84]:
len(hrefs)

534

In [146]:
rating_data = {}

In [147]:
hrefs_done = []
hrefs_no_data = []

In [148]:
# Initialize a progess bar
progress = IntProgress()
progress.max = len(hrefs)
progress.description = '(Init)'
display(progress)
time.sleep(0.25)

for i, href in enumerate(hrefs):
    if href in hrefs_done:
        continue
        
    a = requests.get(url + href)
    soup = bs4.BeautifulSoup(a.text, "lxml")
    name = soup.find("h1", {"id": "HEADING"}).text.strip()
    
    rating_data[name] = {}
    rating_data[name]['url'] = url + href
    try:
        ratings = soup.find("div", {"id": "ratingFilter"}).findAll("li")
    except:
        hrefs_no_data.append(url + href)
        continue

    for row in ratings:
        rating = row.find("div", {"class": "row_label"}).text.strip()
        num = row.find("label").findAll("span")[2].text
        rating_data[name][rating] = int(num.rstrip('\n'))
    
    hrefs_done.append(href)
    
    # change the value of progress bar
    progress.value = i + 1
    progress.description = str(i+1)
    
progress.description = '(Done)'

In [154]:
# number of hotels that have no data
len(hrefs_no_data)

205

In [153]:
# export to .xls
import xlwt
w = xlwt.Workbook(encoding='utf-8')
ws = w.add_sheet('sheet0')
style0 = xlwt.XFStyle()
rating_types = ['Excellent', 'Very good', 'Average', 'Poor', 'Terrible']
for col, rating_type in enumerate(rating_types):
    ws.write(0, col+2, rating_type)
row = 1
for hotel in rating_data:
    ws.write(row, 0, hotel)
    ws.write(row, 1, rating_data[hotel]['url'])
    
    if rating_data[hotel]['url'] in hrefs_no_data:
        row += 1
        continue
    for col, rating_type in enumerate(rating_types):
        ws.write(row, col+2, rating_data[hotel][rating_type])
    row += 1

w.save(r'/Users/hyde/Desktop/Zhuhai_hotels.xls')